### Download wiki chinese corpus and extract

Method 1：wikiextractor https://rnnnnn.medium.com/如何使用中文維基百科-b9230af9d2ed

[use Terminal]
0. brew install wget  <若無wget，請先下載>
0. 選擇儲存位置 cd [path]
1. wget https://dumps.wikimedia.org/zhwiki/latest/zhwiki-latest-pages-articles.xml.bz2   #下載語料
2. git clone https://github.com/attardi/wikiextractor.git wikiextractor   #下載wikiextractor
3. cd wikiextractor #指定路徑到wikiextractor資料夾內
4. python3 setup.py install
5. python3 WikiExtractor.py -b 1024M -o extracted zhwiki-latest-pages-articles.xml.bz2  #把WikiExtractor.py內from .extractor改為 from extractor
6. 處理完成會存在資料夾中：./extracted/AA/wiki_00 & wiki_01

In [1]:
import os
os.getcwd() # 返回目前所在路徑

'/home/jovyan/OSINT/code'

In [2]:
import pandas as pd
import numpy as np
import gensim
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
from gensim.models.word2vec import Word2Vec

In [3]:
main_dir = '/home/jovyan/OSINT'
file_dir = 'Corpus/chinese_corpus'

### Process Wiki Corpus (嘗試做文本分類，似乎不合適)

In [39]:
corpus = pd.read_csv(f'{main_dir}/{file_dir}/wiki_00', delimiter = '\t', header = None)

In [40]:
corpus[:20]

,0
0,"<doc id=""13"" url=""https://zh.wikipedia.org/wik..."
1,数学
2,数学，是研究數量、结构、变化以及空间等概念的一門学科，从某种角度看屬於形式科學的一種。數學利...
3,基礎數學的知識與運用是個人與團體生活中不可或缺的一環。對數學基本概念的完善，早在古埃及、美索...
4,今日，數學使用在一些不同的領域中，包括科學、工程、醫學、經濟學和金融學等。數學對這些領域的應...
5,詞源.
6,西方语言中“數學”（）一詞源自於古希臘語的（），其有“學習”、“學問”、“科學”，以及另外還...
7,汉字表示的「數學」一詞大約产生于中国宋元時期。多指象數之學，但有時也含有今天上的數學意義，例...
8,历史.
9,數學有着久遠的歷史。它被認為起源於人類早期的生產活動：中國古代的六艺之一就有「數」，數學一詞...


In [7]:
# find each documents' title (原想用於文本分類，但似乎不適合)
find_title = corpus[0].str.contains('<doc ') == True
df = corpus[find_title]
title = df[0].str.split(" ").str.get(-1).str.split('"').str.get(1)
title

0                  数学
57                 哲学
193                文學
227                历史
282             计算机科学
              ...    
7797060       小行星9799
7797065       小行星9800
7797070       小行星9801
7797075       枯井惨案纪念碑
7797079    公民1985行動聯盟
Name: 0, Length: 1129894, dtype: object

In [8]:
ws_driver = CkipWordSegmenter(level=3)

Downloading:   0%|          | 0.00/804 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/388M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [58]:
text = ['数学，是研究數量、结构、变化以及空间等概念的一門学科，从某种角度看屬於形式科學的一種。數學利用抽象化和邏輯推理，從計數、計算、量度、對物體形狀及運動的觀察發展而成。數學家們拓展這些概念，以公式化新的猜想，以及從選定的公理及定義出發，嚴謹地推導出一些定理。']

In [61]:
ws = ws_driver(text, use_delim=False)

Inference: 100%|██████████| 1/1 [00:00<00:00,  8.43it/s]


In [62]:
ws

[['数学',
  '，',
  '是',
  '研究',
  '數量',
  '、',
  '结构',
  '、',
  '变化',
  '以及',
  '空间',
  '等',
  '概念',
  '的',
  '一',
  '門',
  '学科',
  '，',
  '从',
  '某',
  '种',
  '角度',
  '看',
  '屬於',
  '形式',
  '科學',
  '的',
  '一',
  '種',
  '。',
  '數學',
  '利用',
  '抽象化',
  '和',
  '邏輯',
  '推理',
  '，',
  '從',
  '計數',
  '、',
  '計算',
  '、',
  '量度',
  '、',
  '對',
  '物體',
  '形狀',
  '及',
  '運動',
  '的',
  '觀察',
  '發展',
  '而',
  '成',
  '。',
  '數學家',
  '們',
  '拓展',
  '這些',
  '概念',
  '，',
  '以',
  '公式化',
  '新',
  '的',
  '猜想',
  '，',
  '以及',
  '從',
  '選定',
  '的',
  '公理',
  '及',
  '定義',
  '出發',
  '，',
  '嚴謹',
  '地',
  '推導出',
  '一些',
  '定理',
  '。']]

In [56]:
corpus_list = list(corpus[0])

In [57]:
corpus_list

['<doc id="13" url="https://zh.wikipedia.org/wiki?curid=13" title="数学">',
 '数学',
 '数学，是研究數量、结构、变化以及空间等概念的一門学科，从某种角度看屬於形式科學的一種。數學利用抽象化和邏輯推理，從計數、計算、量度、對物體形狀及運動的觀察發展而成。數學家們拓展這些概念，以公式化新的猜想，以及從選定的公理及定義出發，嚴謹地推導出一些定理。',
 '基礎數學的知識與運用是個人與團體生活中不可或缺的一環。對數學基本概念的完善，早在古埃及、美索不达米亚及古印度內的古代數學文本便可觀見，而在古希臘那裡有更為嚴謹的處理。從那時開始，數學的發展便持續不斷地小幅進展，至16世紀的文藝復興時期，因为新的科學發現和數學革新兩者的交互，致使數學的加速发展，直至今日。数学并成为許多國家及地區的教育範疇中的一部分。',
 '今日，數學使用在一些不同的領域中，包括科學、工程、醫學、經濟學和金融學等。數學對這些領域的應用通常被稱為應用數學，有時亦會激起新的數學發現，並導致全新學科的發展，例如物理学的实质性发展中建立的某些理论激发数学家对于某些问题的不同角度的思考。數學家也研究純粹數學，就是數學本身的实质性內容，而不以任何實際應用為目標。許多研究雖然以純粹數學開始，但其过程中也發現許多應用之处。',
 '詞源.',
 "西方语言中“數學”（）一詞源自於古希臘語的（），其有“學習”、“學問”、“科學”，以及另外還有個較狹義且技術性的意思－「數學研究」，即使在其語源內。其形容詞（），意思為「和學習有關的」或「用功的」，亦會被用來指「數學的」。其在英语中表面上的複數形式，及在法语中的表面複數形式'，可溯至拉丁文的中性複數'，由西塞罗譯自希臘文複數（），此一希臘語被亚里士多德拿來指「萬物皆數」的概念。",
 '汉字表示的「數學」一詞大約产生于中国宋元時期。多指象數之學，但有時也含有今天上的數學意義，例如，秦九韶的《數學九章》（《永樂大典》記，即《數書九章》也被宋代周密所著的《癸辛雜識》記爲《數學大略》）、《數學通軌》（明代柯尚遷著）、《数学钥》（清代杜知耕著）、《數學拾遺》（清代丁取忠撰）。直到1939年，經過中國數學名詞審查委員會研究“算學”與“數學”兩詞的使用狀況後，確認以“數學”表示今天意義上的數學含義。',
 '历史

### Process Wiki Corpus (使用ckip文本分詞，用於訓練詞向量) --- 耗時需用70.4跑

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import math
import os
import random
import re
import time
import pickle
from collections import Counter
from contextlib import closing
from string import punctuation as enPunc

import torch
import gensim
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker # !pip install -U ckip-transformers
from gensim.models.word2vec import Word2Vec
from opencc import OpenCC  # !pip install opencc
from zhon.hanzi import punctuation as zhPunc  # !pip install zhon

In [ ]:
## 匯入Chinese Corpus -- wiki中文語料
from gensim.corpora import WikiCorpus
file_name = 'III'
input_file = f'/content/drive/MyDrive/{file_name}/zhwiki-latest-pages-articles.xml.bz2'
wiki = WikiCorpus(input_file, dictionary={})
path = f'/content/drive/MyDrive/{file_name}/output_220105.txt'

In [ ]:
# Initialize drivers
ws_driver = CkipWordSegmenter(level=3, device=0)  # device=0 使用GPU
# pos_driver = CkipPosTagger(level=3)
# ner_driver = CkipNerChunker(level=3)

In [2]:
# wiki chinese corpus 分詞
now = time.time()
cc = OpenCC('tw2sp')
corpus = []
i = 109720
savedTime = 0
for index, text in enumerate(wiki.get_texts()):
    if index > i:
        ws = ws_driver(text, use_delim=False)
        for word_seg in ws:
            for idx in range(len(word_seg)):
                corpus.append(word_seg[idx])
        i+=1
    later = time.time()
    if later - now >= 300*(savedTime+1):
        with open(f'/content/drive/MyDrive/{file_name}/ws_log.pkl', 'wb') as f:
            pickle.dump(corpus, f)
            savedTime+=1
            print("file saved!")
            print(i, len(corpus))

In [ ]:
for idx, text in enumerate(wiki.get_texts()):
    if idx == 3127:
        print(text)
        break

In [ ]:
with open(f'/content/drive/MyDrive/{file_name}/ws_log.pkl', 'rb') as f:
    mynewlist = pickle.load(f)
print(len(mynewlist))